In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'window_low_bound': [1, 2], 'window_high_bound': [3, 4, 5, 6],
                            'first_candle': [0.5, 0.6, 0.7, 0.8, 0.9], 'second_candle': [0.5, 0.6, 0.7, 0.8, 0.9]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 200


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [3:24:46<00:00, 61.43s/it]


# Check local statistics

In [3]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
122,Pattern_LinearReg,2,3,0.9,0.7,3,0,164,1.815225,-0.662500,133.696900,-108.650000,1.2470,2.1352,2.3102,2.3935,2.2650,2.2840,2.2705,2.1586,2.1183,2.0728,2.0399,1.9317,1.6693,1.6347,1.6047,1.6108,1.6140,1.5408,1.4737,1.4670,1.4754,1.4595,1.4036,1.3852,-0.32,-0.53,-0.77,-0.70,-0.64,-0.71,-0.70,-0.74,-0.87,-0.89,-0.71,-0.75,-0.63,-0.58,-0.65,-0.82,-0.76,-0.56,-0.56,-0.72,-0.75,-0.51,-0.63,-0.40
123,Pattern_LinearReg,2,3,0.9,0.8,3,0,129,1.807713,-0.651250,104.194913,-84.011250,1.2835,2.3439,2.4530,2.4616,2.3239,2.2895,2.3112,2.2216,2.1392,2.0932,2.0721,1.9381,1.6019,1.5809,1.5434,1.5313,1.5337,1.4666,1.3797,1.3763,1.3911,1.3821,1.3359,1.3314,-0.31,-0.56,-0.82,-0.68,-0.68,-0.73,-0.78,-0.75,-0.89,-0.99,-0.71,-0.70,-0.59,-0.57,-0.65,-0.67,-0.75,-0.49,-0.52,-0.71,-0.72,-0.37,-0.62,-0.37
121,Pattern_LinearReg,2,3,0.9,0.6,3,0,187,1.796396,-0.632917,148.926021,-118.355417,1.2900,2.1446,2.3230,2.4118,2.3153,2.3295,2.2384,2.1228,2.0760,2.0355,1.9924,1.8821,1.6512,1.5996,1.5687,1.5698,1.5757,1.5099,1.4483,1.4376,1.4411,1.4219,1.3752,1.3531,-0.31,-0.52,-0.74,-0.69,-0.65,-0.71,-0.69,-0.66,-0.84,-0.85,-0.67,-0.74,-0.61,-0.52,-0.61,-0.71,-0.75,-0.55,-0.56,-0.71,-0.74,-0.46,-0.52,-0.38
124,Pattern_LinearReg,2,3,0.9,0.9,3,0,83,1.771883,-0.622083,64.066317,-51.632917,1.1618,2.1080,2.2598,2.3124,2.2323,2.1668,2.1966,2.1321,2.0607,1.9741,1.9881,1.8355,1.7345,1.6888,1.5968,1.6054,1.6049,1.5160,1.4265,1.4048,1.4288,1.4079,1.3488,1.3338,-0.23,-0.45,-0.75,-0.64,-0.63,-0.59,-0.63,-0.65,-0.89,-0.91,-0.78,-0.65,-0.59,-0.57,-0.65,-0.73,-0.76,-0.56,-0.56,-0.62,-0.70,-0.37,-0.63,-0.39
118,Pattern_LinearReg,2,3,0.8,0.8,3,0,225,1.753458,-0.641667,169.528125,-144.375000,1.2720,2.2316,2.3772,2.3782,2.2245,2.1249,2.0572,1.9808,1.9878,1.9922,1.9208,1.7588,1.5728,1.5601,1.5398,1.5327,1.4852,1.4653,1.4104,1.4632,1.4676,1.4478,1.4196,1.4125,-0.30,-0.48,-0.75,-0.68,-0.65,-0.70,-0.69,-0.66,-0.79,-0.82,-0.70,-0.56,-0.59,-0.51,-0.61,-0.67,-0.70,-0.54,-0.56,-0.72,-0.91,-0.49,-0.77,-0.55
113,Pattern_LinearReg,2,3,0.7,0.8,3,0,308,1.749083,-0.602917,230.717667,-185.698333,1.2701,2.1826,2.3166,2.3662,2.1954,2.0686,1.9932,1.9471,1.9568,1.9466,1.9032,1.7968,1.6081,1.6093,1.6038,1.5683,1.5370,1.4822,1.4265,1.4606,1.4579,1.4438,1.4221,1.4152,-0.29,-0.50,-0.69,-0.67,-0.64,-0.66,-0.65,-0.63,-0.70,-0.74,-0.67,-0.54,-0.57,-0.51,-0.62,-0.65,-0.66,-0.49,-0.55,-0.72,-0.75,-0.49,-0.65,-0.43
22,Pattern_LinearReg,1,3,0.9,0.7,3,0,178,1.731912,-0.611667,130.280425,-108.876667,1.2036,2.0245,2.1540,2.2479,2.1167,2.1482,2.1403,2.0518,2.0099,1.9511,1.9237,1.8313,1.6166,1.5860,1.5598,1.5677,1.5737,1.5060,1.4437,1.4323,1.4196,1.3990,1.3439,1.3146,-0.31,-0.50,-0.75,-0.70,-0.63,-0.60,-0.68,-0.66,-0.82,-0.85,-0.65,-0.61,-0.59,-0.49,-0.61,-0.73,-0.71,-0.47,-0.55,-0.66,-0.71,-0.46,-0.57,-0.37
120,Pattern_LinearReg,2,3,0.9,0.5,3,0,206,1.728329,-0.586250,150.035808,-120.767500,1.3066,2.1684,2.3242,2.3996,2.2607,2.2144,2.1429,2.0376,1.9890,1.9374,1.8767,1.7514,1.5627,1.4897,1.4701,1.4797,1.4884,1.4339,1.3862,1.3813,1.3852,1.3683,1.3226,1.3029,-0.31,-0.52,-0.74,-0.69,-0.66,-0.71,-0.66,-0.64,-0.72,-0.81,-0.62,-0.66,-0.56,-0.43,-0.53,

# Research parameter influence

In [12]:
param = 'Pattern_window_low_bound'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_window_low_bound,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,1,1.534233,-0.475292,411.625431,-368.622446,79622
1,2,1.554332,-0.492183,409.265800,-367.365058,77148


In [13]:
param = 'Pattern_window_high_bound'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_window_high_bound,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,3,1.625076,-0.527808,225.588708,-187.060817,18621
1,4,1.523453,-0.451042,390.604569,-336.971425,37876
2,5,1.523395,-0.475033,491.862304,-443.211683,47029
3,6,1.505206,-0.481067,533.726879,-504.731083,53244


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [2], 'window_high_bound': [5],
                            'first_candle': [0.6], 'second_candle': [0.8]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)